# MQTT 訂閱測試

本 notebook 包含 MQTT 連接測試和訂閱功能。


In [ ]:
# MQTT 連接測試
import paho.mqtt.client as mqtt
import time
import json

# MQTT 連線設定
MQTT_BROKER = "localhost"  # 或你的 MQTT Broker 地址
MQTT_PORT = 1883
MQTT_TOPIC = "testtopic"  # 測試主題

print("=" * 60)
print("🔍 MQTT 連接測試")
print("=" * 60)

# 測試連接狀態
connection_status = {
    0: "✓ 連接成功",
    1: "✗ 連接被拒絕 - 協議版本不正確",
    2: "✗ 連接被拒絕 - 客戶端 ID 無效",
    3: "✗ 連接被拒絕 - 伺服器不可用",
    4: "✗ 連接被拒絕 - 使用者名稱或密碼錯誤",
    5: "✗ 連接被拒絕 - 未授權"
}

# 連接測試結果
test_results = {
    "connection": False,
    "subscribe": False,
    "publish": False,
    "receive": False
}

# 測試訊息接收標記
test_message_received = False
test_message_content = None

# 創建測試客戶端
test_client = mqtt.Client(mqtt.CallbackAPIVersion.VERSION2)

# 連接回調函數
def on_connect_test(client, userdata, flags, reason_code, properties):
    global test_results
    if reason_code == 0:
        test_results["connection"] = True
        print(f"\n{connection_status[reason_code]}")
        print(f"  Broker: {MQTT_BROKER}:{MQTT_PORT}")
        print(f"  客戶端 ID: {client._client_id.decode()}")
    else:
        test_results["connection"] = False
        status_msg = connection_status.get(reason_code, f"✗ 未知錯誤 (代碼: {reason_code})")
        print(f"\n{status_msg}")

# 訂閱回調函數
def on_subscribe_test(client, userdata, mid, reason_codes, properties):
    global test_results
    test_results["subscribe"] = True
    print(f"✓ 訂閱成功 (mid: {mid})")
    for i, code in enumerate(reason_codes):
        if code == 0:
            print(f"  主題 {i+1}: 訂閱成功 (QoS: {code})")
        else:
            print(f"  主題 {i+1}: 訂閱失敗 (錯誤代碼: {code})")

# 接收訊息回調函數
def on_message_test(client, userdata, message):
    global test_results, test_message_received, test_message_content
    topic = message.topic
    payload = message.payload.decode('utf-8')
    test_message_received = True
    test_message_content = payload
    test_results["receive"] = True
    print(f"\n📨 收到測試訊息:")
    print(f"  主題: {topic}")
    print(f"  內容: {payload}")
    print(f"  QoS: {message.qos}")

# 設置回調函數
test_client.on_connect = on_connect_test
test_client.on_subscribe = on_subscribe_test
test_client.on_message = on_message_test

# 開始測試
print(f"\n1️⃣ 測試連接...")
print(f"   正在連接到 {MQTT_BROKER}:{MQTT_PORT}...")
try:
    test_client.connect(MQTT_BROKER, MQTT_PORT, 60)
    test_client.loop_start()
    time.sleep(2)  # 等待連接
    
    if test_results["connection"]:
        print(f"\n2️⃣ 測試訂閱...")
        print(f"   訂閱主題: {MQTT_TOPIC}")
        test_client.subscribe(MQTT_TOPIC, qos=1)
        time.sleep(1)  # 等待訂閱確認
        
        if test_results["subscribe"]:
            print(f"\n3️⃣ 測試發佈...")
            test_message = f"測試訊息 - {time.strftime('%Y-%m-%d %H:%M:%S')}"
            result = test_client.publish(MQTT_TOPIC, test_message, qos=1)
            result.wait_for_publish()
            test_results["publish"] = True
            print(f"   ✓ 測試訊息已發佈: {test_message}")
            
            print(f"\n4️⃣ 測試接收...")
            print(f"   等待接收測試訊息...")
            time.sleep(2)  # 等待接收訊息
            
            if test_results["receive"]:
                print(f"   ✓ 成功接收測試訊息")
            else:
                print(f"   ⚠️  未收到測試訊息（可能訊息已發送但未訂閱到）")
        
        # 測試結果總結
        print(f"\n" + "=" * 60)
        print("📊 測試結果總結")
        print("=" * 60)
        print(f"連接測試: {'✓ 通過' if test_results['connection'] else '✗ 失敗'}")
        print(f"訂閱測試: {'✓ 通過' if test_results['subscribe'] else '✗ 失敗'}")
        print(f"發佈測試: {'✓ 通過' if test_results['publish'] else '✗ 失敗'}")
        print(f"接收測試: {'✓ 通過' if test_results['receive'] else '✗ 失敗'}")
        
        all_passed = all(test_results.values())
        if all_passed:
            print(f"\n🎉 所有測試通過！MQTT 連接正常。")
        else:
            print(f"\n⚠️  部分測試失敗，請檢查 MQTT Broker 設定。")
        
        # 斷開連接
        print(f"\n正在斷開測試連接...")
        test_client.loop_stop()
        test_client.disconnect()
        print(f"✓ 已斷開連接")
    else:
        print(f"\n❌ 連接失敗，無法繼續測試")
        print(f"   請確認:")
        print(f"   1. MQTT Broker 是否正在運行")
        print(f"   2. Broker 地址是否正確: {MQTT_BROKER}")
        print(f"   3. 端口是否正確: {MQTT_PORT}")
        test_client.loop_stop()
        test_client.disconnect()

except Exception as e:
    print(f"\n❌ 測試過程中發生錯誤: {e}")
    try:
        test_client.loop_stop()
        test_client.disconnect()
    except:
        pass

print(f"\n" + "=" * 60)


Exception in thread paho-mqtt-client-:
Traceback (most recent call last):
  File "/home/pi/.local/share/uv/python/cpython-3.10.19-linux-aarch64-gnu/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/home/pi/.local/share/uv/python/cpython-3.10.19-linux-aarch64-gnu/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/home/pi/Documents/GitHub/pico_202510/.venv/lib/python3.10/site-packages/paho/mqtt/client.py", line 4523, in _thread_main
    self.loop_forever(retry_first_connection=True)
  File "/home/pi/Documents/GitHub/pico_202510/.venv/lib/python3.10/site-packages/paho/mqtt/client.py", line 2297, in loop_forever
    rc = self._loop(timeout)
  File "/home/pi/Documents/GitHub/pico_202510/.venv/lib/python3.10/site-packages/paho/mqtt/client.py", line 1686, in _loop
    rc = self.loop_read()
  File "/home/pi/Documents/GitHub/pico_202510/.venv/lib/python3.10/site-packages/paho/mqtt/client.py", line 2100, in loo

🔍 MQTT 連接測試

1️⃣ 測試連接...
   正在連接到 localhost:1883...

2️⃣ 測試訂閱...
   訂閱主題: testtopic

📊 測試結果總結
連接測試: ✓ 通過
訂閱測試: ✗ 失敗
發佈測試: ✗ 失敗
接收測試: ✗ 失敗

⚠️  部分測試失敗，請檢查 MQTT Broker 設定。

正在斷開測試連接...
✓ 已斷開連接



In [ ]:
import paho.mqtt.client as mqtt
import time
import json

# MQTT 連線設定（與 lesson6_1.ipynb 相同）
MQTT_BROKER = "localhost"  # 或你的 MQTT Broker 地址
MQTT_PORT = 1883
MQTT_TOPIC = "testtopic"  # 訂閱的主題（與發佈主題相同）

# 創建 MQTT 客戶端（使用新的 Callback API 版本 2）
client = mqtt.Client(mqtt.CallbackAPIVersion.VERSION2)

# 連接回調函數（新 API 版本）
def on_connect(client, userdata, flags, reason_code, properties):
    if reason_code == 0:
        print(f"✓ 成功連接到 MQTT Broker: {MQTT_BROKER}:{MQTT_PORT}")
        # 連接成功後訂閱主題
        client.subscribe(MQTT_TOPIC, qos=1)
        print(f"✓ 已訂閱主題: {MQTT_TOPIC}")
    else:
        print(f"✗ 連接失敗，錯誤代碼: {reason_code}")

# 訂閱回調函數（新 API 版本）
def on_subscribe(client, userdata, mid, reason_codes, properties):
    print(f"✓ 訂閱確認 (mid: {mid})")

# 接收訊息回調函數（新 API 版本）
def on_message(client, userdata, message):
    topic = message.topic
    payload = message.payload.decode('utf-8')
    qos = message.qos
    
    print(f"\n{'='*50}")
    print(f"📨 收到新訊息")
    print(f"主題: {topic}")
    print(f"QoS: {qos}")
    print(f"內容: {payload}")
    
    # 嘗試解析 JSON 格式的訊息
    try:
        data = json.loads(payload)
        print(f"📊 JSON 資料解析:")
        for key, value in data.items():
            print(f"  - {key}: {value}")
    except json.JSONDecodeError:
        print(f"ℹ️  純文字訊息")
    print(f"{'='*50}\n")

# 設置回調函數
client.on_connect = on_connect
client.on_subscribe = on_subscribe
client.on_message = on_message

# 連接到 MQTT Broker
print(f"正在連接到 {MQTT_BROKER}:{MQTT_PORT}...")
client.connect(MQTT_BROKER, MQTT_PORT, 60)

# 開始網路循環（阻塞模式，持續監聽訊息）
print(f"\n⏳ 開始監聽主題 '{MQTT_TOPIC}' 的訊息...")
print("提示：執行 lesson6_1.ipynb 來發送測試訊息")
print("按 Ctrl+C 停止監聽\n")

try:
    # 使用 loop_forever() 持續監聽訊息
    client.loop_forever()
except KeyboardInterrupt:
    print("\n\n正在停止訂閱...")
    client.loop_stop()
    client.disconnect()
    print("✓ 已斷開 MQTT 連接")


正在連接到 localhost:1883...

⏳ 開始監聽主題 'testtopic' 的訊息...
提示：執行 lesson6_1.ipynb 來發送測試訊息
按 Ctrl+C 停止監聽

✓ 成功連接到 MQTT Broker: localhost:1883
✓ 已訂閱主題: testtopic
✓ 訂閱確認 (mid: 1)


正在停止訂閱...
✓ 已斷開 MQTT 連接



📨 收到訊息 [testtopic]: Hello MQTT from Streamlit!


## 使用說明

**方式一（阻塞模式）**：執行上面的 cell 後，程式會持續監聽訊息直到按 Ctrl+C 停止。

**方式二（非阻塞模式）**：執行下面的 cell 來使用非阻塞模式，可以更方便地在 notebook 中控制。


In [10]:
# 非阻塞模式訂閱（可選）
# 如果需要非阻塞模式，可以使用以下程式碼

import paho.mqtt.client as mqtt
import time
import json

MQTT_BROKER = "localhost"
MQTT_PORT = 1883
MQTT_TOPIC = "testtopic"

client = mqtt.Client(mqtt.CallbackAPIVersion.VERSION2)

def on_connect(client, userdata, flags, reason_code, properties):
    if reason_code == 0:
        print(f"✓ 成功連接到 MQTT Broker: {MQTT_BROKER}:{MQTT_PORT}")
        client.subscribe(MQTT_TOPIC, qos=1)
        print(f"✓ 已訂閱主題: {MQTT_TOPIC}")
    else:
        print(f"✗ 連接失敗，錯誤代碼: {reason_code}")

def on_subscribe(client, userdata, mid, reason_codes, properties):
    print(f"✓ 訂閱確認 (mid: {mid})")

def on_message(client, userdata, message):
    topic = message.topic
    payload = message.payload.decode('utf-8')
    print(f"\n📨 收到訊息 [{topic}]: {payload}")
    try:
        data = json.loads(payload)
        print(f"   JSON: {data}")
    except:
        pass

client.on_connect = on_connect
client.on_subscribe = on_subscribe
client.on_message = on_message

client.connect(MQTT_BROKER, MQTT_PORT, 60)
client.loop_start()  # 非阻塞模式

print(f"⏳ 開始監聽主題 '{MQTT_TOPIC}' 的訊息...")
print("執行 lesson6_1.ipynb 來發送測試訊息")

# 監聽一段時間（例如 30 秒）
time.sleep(30)

# 停止監聽
client.loop_stop()
client.disconnect()
print("✓ 已停止監聽")

print("這是非阻塞模式的範例程式碼，取消註解後即可使用")


⏳ 開始監聽主題 'testtopic' 的訊息...
執行 lesson6_1.ipynb 來發送測試訊息
✓ 成功連接到 MQTT Broker: localhost:1883
✓ 已訂閱主題: testtopic
✓ 訂閱確認 (mid: 1)
✓ 已停止監聽
這是非阻塞模式的範例程式碼，取消註解後即可使用
